In [138]:
from dataclasses import dataclass
from typing import List, Set, Dict, Tuple, Optional, Union

@dataclass
class Literal:
    name: str
    negate: bool
        
    def __init__(self, name: str):
        self.name = name
        self.negate = True if name.startswith('~') else False
    
    def assign(self, val: bool):
        return not val if self.negate else val
    
    def __repr__(self):
        return f'{self.name}'
    
    def __hash__(self):
        return hash(self.name)

@dataclass
class kCNF:
    k: int
    clauses: List[List[Union[Literal,bool]]]
    literals: Set[Union[Literal,bool]]
        
    def __init__(self, k):
        self.k = k
        self.clauses = []
        self.literals = set()
        
    def add_clause(self, new_clause: List[str]):
        assert(len(new_clause) == self.k)
        self.clauses.append([Literal(c) for c in new_clause])
        for l in new_clause:
            self.literals.add(l.lstrip('~'))
        return self
    
    def assign_literal(self, literal: str, val: bool):
        for c in self.clauses:
            for i,l in enumerate(c):
                if literal == str(l).lstrip('~'):
                    c[i] = l.assign(val)
        return self
    
    def is_satisfied(self):
        assert all(map(lambda c: any([l is True for l in c]), self.clauses)), 'Free clauses exist...'
        return all(map(lambda c: any([l is True for l in c]), self.clauses))
        
    def __repr__(self):
        return ' /\\ '.join(['('+' \\/ '.join(str(x) for x in clause) + ')' for clause in self.clauses])
     

In [139]:
k1 = kCNF(3)
k1.add_clause(['x1', '~x1', 'x2']).add_clause(['x2', '~x3', 'x4'])
k1
k1.assign_literal('x1', True)
k1.assign_literal('x3', False)
print(k1)
k1.is_satisfied()

(True \/ False \/ x2) /\ (x2 \/ True \/ x4)


True

In [171]:
import random
import copy
def sat_solver_naive_sampling(cnf):
    while True:
        assignment = {literal:bool(random.randint(0,1)) for literal in cnf.literals}
        cnf_copy = copy.deepcopy(cnf)
        for lit,val in assignment.items():
            cnf_copy.assign_literal(lit,val)
        if cnf_copy.is_satisfied:
            return assignment
    

In [172]:
import random
k = 3
cnf = kCNF(k)
m = 10000
N = 2000
for i in range(m):
    cnf.add_clause([f'{"~" if random.randint(0,1) else ""}x{random.randint(1,N)}' for _ in range(k)])
#print(cnf)
sat_solver_naive_sampling(cnf)

{'x1879': True,
 'x1225': False,
 'x1635': True,
 'x1494': True,
 'x793': True,
 'x278': True,
 'x1358': True,
 'x344': False,
 'x1311': False,
 'x1411': True,
 'x1680': True,
 'x991': True,
 'x342': False,
 'x1985': True,
 'x485': True,
 'x963': True,
 'x21': False,
 'x1634': True,
 'x1609': False,
 'x980': True,
 'x1335': False,
 'x304': False,
 'x1140': True,
 'x1287': False,
 'x1597': True,
 'x542': False,
 'x749': True,
 'x1282': True,
 'x1988': False,
 'x1886': True,
 'x893': False,
 'x336': True,
 'x655': True,
 'x1154': False,
 'x98': False,
 'x932': True,
 'x124': False,
 'x492': False,
 'x767': False,
 'x788': True,
 'x1436': True,
 'x1270': False,
 'x913': True,
 'x762': True,
 'x1426': False,
 'x1555': False,
 'x727': False,
 'x216': False,
 'x854': True,
 'x839': True,
 'x420': True,
 'x1963': True,
 'x547': False,
 'x1010': False,
 'x545': False,
 'x552': True,
 'x1506': True,
 'x1040': False,
 'x533': False,
 'x395': False,
 'x1625': True,
 'x421': False,
 'x1315': True,

In [ ]:
import random
def sat_solver_LLL(cnf):
    
    